In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
print(pd.__version__)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2.1.4
Num GPUs Available:  0


In [3]:
tf.keras.backend.clear_session()

In [5]:
bid_model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(24 + 1 + 2 + 15,), name="input"),
    tf.keras.layers.Dense(64, activation='relu', name="hidden1"),
    tf.keras.layers.Dense(64, activation='relu', name="hidden2"),
    tf.keras.layers.Dense(11, name="output"),
], name="bid_model")
bid_model.compile(optimizer='adam', loss='mse')

bid_model.summary()

Model: "bid_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden1 (Dense)             (None, 64)                2752      
                                                                 
 hidden2 (Dense)             (None, 64)                4160      
                                                                 
 output (Dense)              (None, 11)                715       
                                                                 
Total params: 7627 (29.79 KB)
Trainable params: 7627 (29.79 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
play_model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(24 + 1 + 2 + 15,), name="input"),
    tf.keras.layers.Dense(64, activation='relu', name="hidden1"),
    tf.keras.layers.Dense(64, activation='relu', name="hidden2"),
    tf.keras.layers.Dense(24, name="output"),
], name="play_model")
bid_model.compile(optimizer='adam', loss='mse')

bid_model.summary()